In [ ]:
import cv2
import numpy as np

In [ ]:
# load Yolo
net = cv2.dnn.readNet("./yolo/Dataset/yolov3.weights","./yolo/Dataset/yolov3.cfg")
classes = []
with open("./yolo/Dataset/coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]


In [ ]:
# load img
cam = cv2.VideoCapture(0)
while True:
    ret,frame = cam.read()
    key_pressed = cv2.waitKey(1)&0xFF
    if ret==False:
        print("Something went wrong")
        continue
    if key_pressed == ord('q'):
        break
    cv2.imshow("Actual-Image", frame)
#     img = cv2.imread("./yolo/Dataset/room_ser.jpg")
    img = frame
    height, width, channels = img.shape
    # detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    # for b in blob:
    #     for i,img_b in enumerate(b):
    #         cv2.imshow(str(i), img_b)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # showing information on the screen

    boxes = []
    confidences = []
    class_ids = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                cv2.circle(img, (center_x, center_y), 10, (0, 255, 0), 2)

                x = int(center_x - w/2)
                y = int(center_y - h/2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    no_boxes_detected = len(boxes)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(no_boxes_detected):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 1)
            cv2.putText(img, label, (x, y+30), font, 3, (0,0,0), 3)
    

    cv2.imshow("Image", img)
cv2.waitKey(0)
cv2.destroyAllWindows()